In [ ]:
import MLJ
import DataFrames as DF
import CSV
using DataFramesMeta
using StatsPlots

In [ ]:
include("PreprocessAerosolData.jl")
include("ActivationEmulatorModels.jl")

In [ ]:
mach = MLJ.machine("emulators/nn_machine2.jls")

In [ ]:
X_test, Y_test = read_aerosol_dataset("datasets/dataset13_test.csv", true)

In [ ]:
pred = MLJ.predict(mach, X_test).act_frac_S

In [ ]:
p1 = scatter(Y_test.act_frac_S, preprocess_aerosol_data(X_test).ARG_act_frac, color=:black, markersize=1,
            markerstrokewidth=0, label=nothing, title="ARG", xlabel="PySDM act frac", ylabel="Predicted act frac")
plot!([0, 1], [0, 1], color=:red, label=nothing)
p2 = scatter(Y_test.act_frac_S, pred, color=:black, markersize=1, markerstrokewidth=0, label=nothing,
            title="Emulator", xlabel="PySDM act frac", ylabel="Predicted act frac")
plot!([0, 1], [0, 1], color=:red, label=nothing)
plot(p1, p2)

In [ ]:
p1 = histogram(preprocess_aerosol_data(X_test).ARG_act_frac - Y_test.act_frac_S, label=nothing, xlabel="predicted - true act frac", title="ARG")
p2 = histogram(pred - Y_test.act_frac_S, label=nothing, xlabel="predicted - true act frac", title="Emulator")
plot(p1, p2)